In [14]:
import requests
from bs4 import BeautifulSoup
import chardet

import pandas as pd
import os
import argparse


#import face_recognition
from PIL import Image
import numpy as np
import pandas as pd
from imutils import face_utils
import cv2

from math import sqrt



import math
import sys
import argparse
from tqdm import tqdm
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt

## 爬立委資料

In [ ]:
url = "https://www.ly.gov.tw/Pages/List.aspx?nodeid=109"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive'
}

response = requests.get(url, headers=headers)

encoding = chardet.detect(response.content)['encoding']
html_content = response.content.decode(encoding , errors='replace')

soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())


In [ ]:
page_lists = []

for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if "/Pages/List.aspx?nodeid=" in href:
        page_lists.append(href)

for page in page_lists:
    print(page)


In [ ]:
link1 = "https://www.ly.gov.tw"
start = 124
end = start + 114

# Lists to store the data
names = []
eng_names = []
partys = []
sessions = []
genders = []
photos = []

# Directory to save images
image_dir = "legislator_images"
os.makedirs(image_dir, exist_ok=True)

for page in page_lists[start:end]:
    url = link1 + page
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract information
    name = soup.find('div', class_='legislatorname').get_text(strip=True)
    eng_name = soup.find('li', text=lambda x: x and '英文姓名' in x).get_text(strip=True).split('：')[1]
    party = soup.find('li', text=lambda x: x and '黨籍' in x).get_text(strip=True).split('：')[1]
    session = soup.find('li', text=lambda x: x and '屆別' in x).get_text(strip=True).split('：')[1]
    gender = soup.find('li', text=lambda x: x and '性別' in x).get_text(strip=True).split('：')[1]
    photo_url = soup.find('img', class_='img-responsive')['src']
    
    # Store the information
    names.append(name)
    eng_names.append(eng_name)
    partys.append(party)
    sessions.append(session)
    genders.append(gender)
    photos.append(photo_url)
    
    # Save the photo
    
    photo_filename = os.path.join(image_dir, photo_url.split('/')[-1])    
    with open(photo_filename, 'wb') as f:
        f.write(requests.get(link1 + photo_url).content)
        print(link1 + photo_url)
    

# Create a DataFrame
df = pd.DataFrame({
    'Name': names,
    'English Name': eng_names,
    'Party': partys,
    'Session': sessions,
    'Gender': genders,
    'Photo Filename': [photo.split('/')[-1] for photo in photos]
})

# Save the DataFrame to a CSV file
df.to_csv('legislators_info.csv', index=False)

print("Data extraction and image saving complete.")

## 動態生成分類字典，處理所有出現的黨派和性別組合

In [ ]:
import os
import pandas as pd
import shutil


# 讀取CSV文件，Name, English Name, Session, Gender, Photo Filename, Party
df = pd.read_csv('legislators_info.csv')

image_dir = "legislator_images"

classified_images = {}

# 分類圖像
for _, row in df.iterrows():
    party = row['Party']  
    gender = row['Gender']
    photo_filename = row['Photo Filename']
    
    category = f"{party}_{gender}"
    
    if category not in classified_images:
        classified_images[category] = []
    
    image_path = os.path.join(image_dir, photo_filename)
    if os.path.exists(image_path):
        classified_images[category].append(image_path)

aligned_faces_dir = "aligned_faces"
os.makedirs(aligned_faces_dir, exist_ok=True)

# 複製圖片到相關目錄
for category, images in classified_images.items():
    category_dir = os.path.join(aligned_faces_dir, category)
    os.makedirs(category_dir, exist_ok=True)
    
    for image_path in images:
        # 複製img
        image_name = os.path.basename(image_path)
        destination_path = os.path.join(category_dir, image_name)
        shutil.copy(image_path, destination_path)  # 使用 shutil.copy 进行复制